In [1]:
file = spark.sparkContext.textFile('file:///C:/Users/Ashwini/Desktop/XYonTimeArrPerf')

In [6]:
file.count()

19914

In [2]:
file = file.map(lambda line:line.split())

In [3]:
file1 = file.filter(lambda row:len(row)==4).map(lambda tuple:(tuple[0],tuple[1],tuple[2],float(tuple[3])))

In [17]:
file1.count()

19466

In [4]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('aviation')

In [10]:
#query = "INSERT INTO airport_srcdest_arrival (src, dest, arr_delay) VALUES (?, ?, ?)"
#prepared = session.prepare(query)

In [ ]:
#session.execute(prepared, (row.select('src'), row.select('dest'),row.select('arr_delay')))

In [5]:
from pyspark.sql import Row
rdd3 = file1.map(lambda row: Row(src=row[0], dest=row[1], carrier=row[2], arr_delay=row[3]))


In [9]:
df = spark.createDataFrame(rdd3)

In [21]:
df.show()

+-------------------+-------+----+---+
|          arr_delay|carrier|dest|src|
+-------------------+-------+----+---+
|               23.0|     AA| ALB|ABE|
|  6.266970753957606|     DL| ATL|ABE|
| 3.4953071672354947|     EA| ATL|ABE|
| 12.214985308521058|     EV| ATL|ABE|
|   8.44742857142857|     OH| ATL|ABE|
|  5.929155313351498|     EA| AVP|ABE|
|                0.0|     EV| AVP|ABE|
|               20.5|     OH| AVP|ABE|
|-1.6032689450222883|     US| AVP|ABE|
|                1.0|     AA| BDL|ABE|
|               -3.0|     OO| BHM|ABE|
|               61.0|     OH| BWI|ABE|
| 10.809917355371901|     PI| BWI|ABE|
| 1.2485436893203883|     TW| BWI|ABE|
|  4.828444936358605|     US| BWI|ABE|
|-2.1604059106284494|     XE| CLE|ABE|
| 0.8229916067146283|     US| CLT|ABE|
|               6.98|     YV| CLT|ABE|
| 2.0990654205607475|     DL| CVG|ABE|
|0.20199778024417314|     EV| CVG|ABE|
+-------------------+-------+----+---+
only showing top 20 rows



In [22]:
pip install cassandra-driver


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from cassandra.cluster import Cluster

In [7]:
cluster = Cluster(['127.0.0.1'])  # provide contact points and port
session = cluster.connect('aviation')

In [40]:
#rows = session.execute('select * from airport_srcdest_arrival;')

In [41]:
for row in rows:
    print(row)

Row(src='APT', dest='SDF', arr_delay=Decimal('-9.8'))


In [52]:
query = "INSERT INTO airport_srcdest_arrival (src, dest, arr_delay) VALUES (?, ?, ?)"


In [30]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [54]:
prepared = session.prepare(query)

In [55]:
session.execute(prepared, ("DAD", "SDA", -2.34))

In [57]:
rows = session.execute('select * from airport_srcdest_arrival;')
for row in rows:
    print(row)

Row(src='APT', dest='SDF', arr_delay=Decimal('-9.8'))
Row(src='DAD', dest='SDA', arr_delay=Decimal('-2.339999999999999857891452847979962825775146484375'))


In [16]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [62]:
ds = sqlContext.read \
  .format('org.apache.spark.sql.cassandra') \
  .options(table='airport_srcdest_arrival', keyspace='aviation') \
  .load()

In [63]:
ds.show()

+---+----+--------------------+
|src|dest|           arr_delay|
+---+----+--------------------+
|ABQ| DFW|4.222648752399232000|
|ABQ| DFW|5.601449275362318000|
+---+----+--------------------+



In [10]:
df.write\
.format("org.apache.spark.sql.cassandra")\
.mode('append')\
.options(table = "airport_src_dest_arrival", keyspace = "aviation")  \
.save()